# This is Image Classification for Cute

Get the current working directory

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['image-classification', 'vgg16-weights']


In [2]:
PATH=os.getcwd()
print(PATH)

/kaggle/working


In [3]:
PATH="../input/image-classification/image/Image/10_categories"

In [4]:
print(os.listdir(PATH))

['car_side', 'watch', 'bonsai', 'grand_piano', 'BACKGROUND_Google', 'airplanes', 'Faces_easy', 'Motorbikes', 'Leopards', 'Faces']


In [5]:
img_rows=224
img_cols=224
num_channel=3

num_epoch=3
batch_size=32

img_data_list=[]
classes_names_list=[]
data_dir_list=os.listdir(PATH)

In [6]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(PATH+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)

Loading images from car_side folder

Loading images from watch folder

Loading images from bonsai folder

Loading images from grand_piano folder

Loading images from BACKGROUND_Google folder

Loading images from airplanes folder

Loading images from Faces_easy folder

Loading images from Motorbikes folder

Loading images from Leopards folder

Loading images from Faces folder



In [7]:
num_classes = len(classes_names_list)
print(num_classes)

10


In [8]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [9]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [10]:
print (img_data.shape)
print(num_of_samples)
print(input_shape)

(3725, 224, 224, 3)
3725
(224, 224, 3)


In [11]:
classes = np.ones((num_of_samples,), dtype='int64')

classes[0:239]=0
classes[239:367]=1
classes[367:490]=2
classes[490:690]=3
classes[690:1490]=4
classes[1490:1958]=5
classes[1958:2393]=6
classes[2393:2828]=7
classes[2828:3626]=8
classes[3626:]=9


In [12]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

Using TensorFlow backend.


In [13]:
print(classes.shape)
print(classes)


(3725, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [14]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [16]:
y_test.shape

(745, 10)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [18]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

In [20]:
num_epoch=12

In [21]:
from keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.5, 
    zoom_range=0.4, 
    rescale=1./255,
    vertical_flip=True, 
    validation_split=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True) 

In [22]:
#TRN_AUGMENTED = os.path.join(PATH  , 'Trn_Augmented_Images')
#TST_AUGMENTED = os.path.join(PATH  , 'Tst_Augmented_Images')

In [23]:
train_generator = data_gen.flow_from_directory(
        PATH,
        target_size=(img_rows, img_cols), 
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb', 
        shuffle=True)

 #       save_to_dir=TRN_AUGMENTED, 
 #       save_prefix='TrainAugmented', 
 #       save_format='png', 
 #       subset="training")

Found 3724 images belonging to 10 classes.


In [24]:
train_generator.class_indices

{'BACKGROUND_Google': 0,
 'Faces': 1,
 'Faces_easy': 2,
 'Leopards': 3,
 'Motorbikes': 4,
 'airplanes': 5,
 'bonsai': 6,
 'car_side': 7,
 'grand_piano': 8,
 'watch': 9}

In [25]:
test_generator = data_gen.flow_from_directory(
        PATH,
        target_size=(img_rows, img_cols),
        batch_size=32,
        class_mode='categorical',
        color_mode='rgb', 
        shuffle=True, 
        seed=None)

#        save_to_dir=TST_AUGMENTED, 
#        save_prefix='TestAugmented', 
#        save_format='png',
#        subset="validation")

Found 3724 images belonging to 10 classes.


In [26]:
test_generator.class_indices

{'BACKGROUND_Google': 0,
 'Faces': 1,
 'Faces_easy': 2,
 'Leopards': 3,
 'Motorbikes': 4,
 'airplanes': 5,
 'bonsai': 6,
 'car_side': 7,
 'grand_piano': 8,
 'watch': 9}

In [27]:
model.fit_generator(train_generator, epochs=num_epoch, validation_data=test_generator,steps_per_epoch = len(X_train)/batch_size,validation_steps=len(X_test)/batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/12
94/93 [==============================] - 56s 597ms/step - loss: 2.1697 - acc: 0.2839 - val_loss: 1.8436 - val_acc: 0.4531
Epoch 2/12
94/93 [==============================] - 49s 518ms/step - loss: 1.6486 - acc: 0.4511 - val_loss: 1.3899 - val_acc: 0.5872
Epoch 3/12
94/93 [==============================] - 49s 519ms/step - loss: 1.4037 - acc: 0.5307 - val_loss: 1.0928 - val_acc: 0.6289
Epoch 4/12
94/93 [==============================] - 48s 514ms/step - loss: 1.2986 - acc: 0.5644 - val_loss: 0.9301 - val_acc: 0.6680
Epoch 5/12
94/93 [==============================] - 48s 508ms/step - loss: 1.1947 - acc: 0.5985 - val_loss: 1.0107 - val_acc: 0.6471
Epoch 6/12
94/93 [==============================] - 48s 516ms/step - loss: 1.1591 - acc: 0.6320 - val_loss: 0.9284 - val_acc: 0.7070
Epoch 7/12
94/93 [==============================] - 48s 513ms/step - loss: 1.1161 - acc: 0.6363 - val_loss: 0.8062 - val_acc: 0.7161
Epoch 8/12
94/93 [===

In [28]:
model.evaluate_generator(train_generator,steps = len(X_train)/batch_size,verbose=1)

94/93 [==============================] - 40s 428ms/step


[0.9613352602265446, 0.7028112450597117]

In [29]:
Y_pred = model.predict(X_test)
print(Y_pred)
Y_train_result = model.predict(X_train)
print(Y_train_result)

[[2.2713083e-01 8.0612898e-02 4.3078926e-03 ... 6.4196871e-03
  3.2189243e-02 2.4310574e-01]
 [2.3544773e-01 3.5610878e-01 3.8375934e-03 ... 5.6239096e-03
  1.1530529e-02 1.2669747e-01]
 [2.1850845e-01 2.9675609e-02 1.6193866e-03 ... 3.5028593e-03
  1.8292662e-02 3.2964915e-01]
 ...
 [1.0514811e-01 5.6129828e-02 1.0421747e-02 ... 3.6626149e-04
  2.2161307e-02 1.8632211e-01]
 [5.4655021e-01 4.3969404e-02 6.6151371e-04 ... 3.1659788e-08
  3.5682222e-06 4.0849230e-01]
 [8.2392665e-04 2.8876022e-07 2.3655806e-09 ... 1.0115013e-08
  5.7426252e-04 2.6890347e-03]]
[[2.49625318e-05 1.53155572e-10 3.59275123e-14 ... 3.67045424e-13
  4.60792171e-06 8.58438652e-05]
 [3.46100181e-02 2.39063259e-02 1.56059302e-03 ... 6.68716490e-01
  1.87495002e-03 2.75037140e-02]
 [1.31479442e-01 5.18393063e-06 2.37264430e-07 ... 6.63837341e-07
  2.38967245e-03 6.44101024e-01]
 ...
 [2.01428175e-01 3.43344808e-02 4.49256942e-04 ... 8.00235488e-04
  1.10783977e-02 1.19015664e-01]
 [4.03131053e-05 5.66541085e-11 9.6

In [30]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
y_train_result = np.argmax(Y_train_result, axis=1)
print(y_train_result)

[4 1 4 4 4 4 4 2 4 5 4 4 0 4 4 4 4 0 4 4 5 5 4 5 4 5 0 4 5 5 4 5 4 4 5 4 4
 5 0 4 4 4 4 4 4 4 4 4 3 0 4 4 4 4 4 4 0 4 4 4 0 4 0 4 5 5 0 4 4 0 4 0 4 5
 5 5 0 5 0 0 4 4 4 4 1 4 4 4 9 4 0 4 4 5 4 4 5 5 4 4 5 4 5 0 4 4 4 5 4 4 4
 4 4 5 4 4 4 0 4 4 1 4 4 4 4 7 4 5 5 4 4 5 5 4 4 1 4 4 9 0 4 4 4 4 0 7 4 5
 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 5 4 4 4 5 0 4 4 4 4 5 4 4 1 4 4 4 0 4 0 4 4
 4 4 0 4 4 5 4 4 4 0 5 4 4 1 4 4 4 5 4 4 4 4 5 5 5 4 5 4 5 5 4 4 4 4 5 2 9
 4 4 4 4 4 4 4 4 4 3 9 4 4 5 4 7 0 0 5 7 5 4 1 7 4 4 4 5 0 4 4 4 4 4 0 4 4
 4 0 5 4 4 0 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 5 4 5 0 4 4 4 9 7 5 4 5 4 4 4 4
 5 5 4 4 4 0 5 5 4 4 4 9 4 4 4 4 7 4 5 4 4 4 4 1 4 4 5 0 4 4 4 4 4 4 3 4 4
 5 4 4 4 4 4 4 0 4 4 9 0 4 5 4 0 1 5 4 4 0 4 5 4 7 4 4 4 4 4 4 5 4 4 4 4 0
 5 4 1 4 4 5 4 5 5 5 4 4 5 4 4 4 4 4 0 4 0 4 1 4 0 5 0 4 7 9 4 0 4 0 4 5 4
 4 2 4 4 4 4 4 4 4 4 1 4 4 4 5 4 4 4 9 1 4 4 5 0 5 4 4 7 4 4 2 4 4 0 5 4 5
 2 4 5 4 4 4 4 4 5 0 4 5 4 4 4 4 0 0 4 4 5 4 5 4 0 5 4 4 4 4 0 0 0 4 0 4 9
 4 7 4 0 1 4 4 5 4 5 0 0 

In [31]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,accuracy_score


In [32]:

y_true=np.argmax(y_test,axis=1)
y_true_train=np.argmax(y_train,axis=1)


In [33]:
print(classification_report(y_pred,y_true))
print(classification_report(y_train_result,y_true_train))
print("test accuracy = ",accuracy_score(y_pred,y_true))
print("validation accuracy = ",accuracy_score(y_train_result,y_true_train))

              precision    recall  f1-score   support

           0       0.05      0.03      0.04       103
           1       0.00      0.00      0.00        21
           2       0.00      0.00      0.00         7
           3       0.02      0.25      0.04         4
           4       0.34      0.11      0.16       463
           5       0.47      0.42      0.44       112
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00        19

   micro avg       0.13      0.13      0.13       745
   macro avg       0.09      0.08      0.07       745
weighted avg       0.29      0.13      0.17       745

              precision    recall  f1-score   support

           0       0.07      0.03      0.04       399
           1       0.00      0.00      0.00        94
           2       0.00      0.00      0.00        34
           3       0.01 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [34]:
#from IPython.display import Image
#Image(filename='vgg16.png')

In [35]:
from tensorflow.keras.layers import Input, Dense

In [36]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(img_rows, img_cols, num_channel))

In [37]:
from tensorflow.keras.applications.vgg16 import VGG16

print(os.listdir("../input/vgg16-weights"))

['vgg16_weights_tf_dim_ordering_tf_kernels.h5']


In [38]:
from keras import applications
model = VGG16(input_tensor=image_input, include_top=True, weights='../input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5', )

In [39]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [40]:
from tensorflow.keras.models import Model

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [41]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

In [42]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [43]:
num_epoch = 100
custom_vgg_model.fit(X_train, y_train, batch_size=512, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 2980 samples, validate on 745 samples
Epoch 1/100
2980/2980 [==============================] - 25s 8ms/sample - loss: 2.5915 - acc: 0.1893 - val_loss: 2.1935 - val_acc: 0.3611
Epoch 2/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 2.0670 - acc: 0.2990 - val_loss: 1.7988 - val_acc: 0.4000
Epoch 3/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 1.7491 - acc: 0.4775 - val_loss: 1.5573 - val_acc: 0.4295
Epoch 4/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 1.5026 - acc: 0.5121 - val_loss: 1.4651 - val_acc: 0.5195
Epoch 5/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 1.3825 - acc: 0.5362 - val_loss: 1.3093 - val_acc: 0.5651
Epoch 6/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 1.3076 - acc: 0.5443 - val_loss: 1.2614 - val_acc: 0.5638
Epoch 7/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 1.2256 - acc: 0.5876 - val_loss: 1.2089 - val

2980/2980 [==============================] - 12s 4ms/sample - loss: 0.6979 - acc: 0.7017 - val_loss: 0.7923 - val_acc: 0.6403
Epoch 60/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 0.6980 - acc: 0.7070 - val_loss: 0.8255 - val_acc: 0.6322
Epoch 61/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 0.7161 - acc: 0.6953 - val_loss: 0.7999 - val_acc: 0.6134
Epoch 62/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 0.7200 - acc: 0.6862 - val_loss: 0.8140 - val_acc: 0.6282
Epoch 63/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 0.7118 - acc: 0.6980 - val_loss: 0.8013 - val_acc: 0.6081
Epoch 64/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 0.6961 - acc: 0.6997 - val_loss: 0.8112 - val_acc: 0.6309
Epoch 65/100
2980/2980 [==============================] - 12s 4ms/sample - loss: 0.6935 - acc: 0.7000 - val_loss: 0.8168 - val_acc: 0.6268
Epoch 66/100
2980/2980 [================

In [44]:
from keras.models import load_model

custom_vgg_model.save('Image_Class_Cute5.h5')

In [45]:
(loss, accuracy) = custom_vgg_model.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

2980/2980 [==============================] - 9s 3ms/sample - loss: 0.6288 - acc: 0.7262
[INFO] loss=0.6288, accuracy: 72.6174%


In [46]:
Y_train_pred = custom_vgg_model.predict(X_test)

In [47]:
y_train_pred = np.argmax(Y_train_pred, axis=1)
print(y_train_pred)

[8 8 1 0 8 8 2 6 7 8 7 7 3 4 6 7 7 4 7 7 4 4 4 4 6 4 4 0 4 5 7 4 6 7 4 7 6
 4 8 4 4 6 7 7 7 4 0 4 4 4 0 6 7 4 4 7 8 2 8 7 4 7 0 4 8 4 6 6 6 4 9 8 7 4
 4 4 8 4 3 4 0 0 4 0 8 8 6 2 8 3 4 4 6 4 0 7 4 4 7 7 8 3 4 4 8 7 6 4 8 6 6
 1 6 8 7 4 7 6 7 0 8 7 8 8 4 0 7 6 4 4 1 4 4 7 4 8 7 7 8 1 0 6 0 7 1 0 7 4
 7 0 4 0 6 4 6 6 6 1 6 4 6 4 8 4 7 7 4 4 4 4 3 3 4 4 4 6 6 7 4 7 4 6 4 7 0
 2 7 9 7 8 1 4 8 3 2 4 0 6 9 4 3 0 5 8 4 8 8 4 8 4 8 4 8 4 4 7 7 8 6 8 6 0
 2 3 7 8 7 4 6 6 0 5 0 2 7 4 6 0 8 4 5 0 2 2 8 4 7 6 6 4 4 7 8 9 0 4 4 4 7
 8 4 4 4 3 2 0 8 0 8 6 7 4 7 4 8 4 7 6 0 7 5 7 4 4 7 6 8 2 0 8 7 6 7 7 8 7
 4 8 3 0 7 4 4 8 1 7 0 2 7 0 8 4 0 0 4 8 7 7 2 5 8 5 4 4 4 1 4 8 0 4 4 3 7
 4 4 8 7 7 2 3 6 1 6 4 4 6 4 7 4 8 4 2 7 4 7 4 7 4 3 3 8 7 3 8 5 8 7 4 0 4
 4 7 6 2 7 8 8 5 8 4 7 7 4 0 7 8 7 4 2 4 4 8 8 7 4 4 0 8 3 8 4 4 7 3 8 4 4
 8 6 7 2 2 3 1 7 7 8 8 0 7 8 4 4 4 8 0 8 8 8 4 6 4 8 1 0 7 5 6 7 4 6 6 2 4
 6 0 4 4 8 4 7 7 4 4 2 4 6 8 0 7 9 2 3 2 4 7 4 8 4 4 3 7 4 8 8 4 3 4 4 7 6
 6 0 8 4 6 6 7 4 8 1 4 4 

In [48]:
from sklearn.metrics import classification_report,accuracy_score
print("test accuracy = ",accuracy_score(y_train_pred ,np.argmax(y_test, axis=1),))
#print("test accuracy = ",accuracy_score(y_pred,y_true))

test accuracy =  0.6442953020134228
